In [1]:
import os
import numpy as np

In [2]:
def read_fc_data_file(fc_data_file):
    ''' Read an existing functional connectivity data file that also contains labels and demongraphics,
    or create it if it doesn't exist. Returns the data file.
    Args:
        fc_data_file (str): The complete path to the fc data file  
    '''
    import pandas as pd
    if not os.path.exists(fc_data_file):
        import create_fc_fisher_z_csv_file as makedata
        subjDF = makedata.read_data()
        fcData = makedata.read_fc_data(subjDF)
        fcData.to_csv(path_or_buf=fc_data_file, index=True)
    else:
        fcData = pd.read_csv(fc_data_file, index_col=0)
    fcData['func_perc_quart'] = pd.qcut(fcData['func_perc_fd'], q=4, labels=False)
    return fcData

In [3]:
def ppc_fc_data(fcData, age_range, motion_threshold, 
                labels_col = 'DX_GROUP', strata_cols = ['DX_GROUP', 'func_perc_quart', 'SEX'], 
                agecol='AGE_YRS', motioncol='func_perc_fd'):
    ''' Takes an fc file with functional connectivity columns as well as demographics and returns
    strata, labels, and features with imputed values.
    Args:
        fcData (pandas.DataFrame): The fc data file
        age_range (tuple): lower and upper age range for sample
        motion_threshold (int or float): sample will contain rows with func_perc_fd <= motion_threshold
        strata_cols (list of str): columns used to stratify
    '''
    from sklearn.preprocessing import Imputer
    fcData_threshed = fcData.query(agecol + " >= " + str(age_range[0]) + " & " + agecol + "<= " + str(age_range[1]) + " & " + motioncol + " <= " + str(motion_threshold))
    strata = fcData_threshed.groupby(strata_cols).grouper.group_info[0]
    labels = fcData_threshed[labels_col].as_matrix()
    features = fcData_threshed.loc[:,'#2001_#2002':'#9160_#9170']
    features_imputed = Imputer(missing_values='NaN', strategy='mean', axis=0).fit_transform(features)
    return strata, labels, features_imputed

In [4]:
def trainModel(data, labels, strata, modelDir = None, fname_prefix = None, classifier = 'svc',  cv = True, cvmethod = 'sss', n_iter = 10, k = 10, sparse = True, saveData = True, n_jobs=1):
    '''Applies classifier to predict labels from data, stratified by strata, and returns the resulting model.
    Optionally saves the model and coefficients.
    Args:
        data (numpy.ndarray): features to use in prediction
        labels (numpy.ndarray): labels to predict
        strata (numpy.ndarray): labels to use for stratification
        classifier (str): classifier to use. currently only svc is implemented
        modelDir (str): where to save model and csv files
        fname_prefix (str): what to attach to the beginning of the filename
        cv (bool): implement cross validation? currently this must be true
        cvmethod (str): cross validation method, 'sss' for StratifiedShuffleSplit or 'skf' for StratifiedKFold.
        n_iter (int): number of times to iterate the StratifiedShuffleSplit; default true
        k (int): if classifier is StratifiedKFold, number of folds; default true
        sparse (bool): whether to use l1 (sparse) regularization or l2; default True
        saveData (bool): save the results to model directory? default True
        n_jobs (int): number of jobs to pass to cv
    '''
    if sparse:
        penalty = 'l1'
    else:
        penalty = 'l2'
    if classifier == 'svc':
        from sklearn.svm import LinearSVC
        algorithm = LinearSVC(penalty = penalty)
        if penalty == 'l1':
            algorithm.set_params(dual=False)
    if cv:
        from sklearn.grid_search import GridSearchCV
        paramsToSearch = []        
        paramsToSearch.append({'C': [.001,.005,.01,.1,1,10]})
        if cvmethod == 'sss':
            from sklearn.cross_validation import StratifiedShuffleSplit
            cvalgorithm = StratifiedShuffleSplit(strata, n_iter = n_iter, test_size = .3)
        if cvmethod == 'skf':
            from sklearn.cross_validation import StratifiedKFold
            cvalgorithm = StratifiedKFold(strata, n_folds = n_folds, shuffle = True)
        clf=[]
        clf = GridSearchCV(algorithm, 
                           paramsToSearch, 
                           cv=cvalgorithm,
                           n_jobs=n_jobs)
    else:
        #Defaults and train model
        clf = algorithm
    clf.fit(data, labels)
    
    #Get training accuracy
    from sklearn.metrics import accuracy_score
    trainingPredictions = clf.predict(data)
    accuracy = accuracy_score(trainingPredictions, labels)
    print "Training accuracy: %f" % accuracy

    if cv:
        #Extract tuned model weights
        modelWeights = clf.best_estimator_.coef_[0]
        modelIntercept = clf.best_estimator_.intercept_
    else:
        #Extract deafult model weights
        modelWeights = clf.coef_[0]
        modelIntercept = clf.intercept_
    
    if saveData:
        if (not modelDir) | (not fname_prefix):
            raise TypeError('Model dir or file name not specified')
        print "Saving model weights, training accuracy, and model"
        from sklearn.externals import joblib
        #Write intercept + model weights to csv 
        if not os.path.exists(modelDir):
            print ("{} does not exist; creating ...".format(modelDir))
            os.makedirs(modelDir)
        fileName = os.path.join(modelDir, fname_prefix)
        print ("Writing data to {}*".format(fileName))
        np.savetxt(fileName + '_Weights.csv',np.concatenate([modelIntercept,modelWeights]), delimiter=',', newline=os.linesep)
        np.savetxt(fileName + '_TrainAcc.csv',np.array([accuracy]), delimiter=',',newline=os.linesep)
        joblib.dump(clf, fileName + '_Model.pkl')
        
    return (clf)

In [5]:
def testModel(data, labels, clf = None, modelDir = None, fname_prefix = None, outputDir = None, saveData=True):
    
    ''' Test a linear classifier by loading a fitted model and returning predictions on given data.
    Args:
        data (ndarray): A data matrix organized as nsamples x nfeatures 
        labels (ndarray): A 1d labels array same length as nsamples  
        clf (sklearn fit object): If not specified, fucntion will try to load using modelDir and fname_prefix
        modelDir (str): directory from which to load pickled model files
        fname_prefix (str): filename prefix used for model files
        outDir (str): directory to write csv file with testing accuracy and predictions
        saveData (bool; optional): whether to actually save csv or just return model object; 
            default True
    '''

    from sklearn.externals import joblib
    from sklearn.metrics import accuracy_score

    if not clf:
        if (not modelDir) | (not fname_prefix):
            raise TypeError('No clf provided and Model dir or file name not specified')
        modelPath = os.path.join(modelDir, fname_prefix)
        #If model doesn't exist use csv with coefs - TODO
        clf = joblib.load(modelPath + '_Model.pkl')
    predictions = clf.predict(data)

    #Compute accuracy on test data
    accuracy = accuracy_score(predictions,labels)
    print "Testing accuracy: %f" % accuracy

    if saveData:
        from sklearn.externals import joblib
        if (not outputDir) | (not fname_prefix):
            raise TypeError('Output dir or file name not specified')
        if not os.path.exists(outputDir):
            print ("{} does not exist; creating ...".format(outputDir))
            os.makedirs(outputDir)
        outPath = os.path.join(outputDir, fname_prefix)
        print "Saving test accuracy and predictions to {}*".format(outPath)
        #Save accuracy
        np.savetxt(outPath + '_TestAcc.csv', np.array([accuracy]), delimiter=',',newline=os.linesep)
        #Save predictions
        np.savetxt(outPath + '_Predictions.csv',predictions, delimiter=',',newline=os.linesep)

    return accuracy 


In [56]:
oos_iter = 10
sss_n_iter=10
skf_n_folds=3
age_range=(6,18)
motion_threshold=5
sample_size=30
classifier = 'svc'
cvmethod = 'sss'
modelDir = './cv_models/test'
outputDir = './cv_output/test'
fname_prefix = "{}_{}_mt{}_n{}".format(cvmethod, classifier, motion_threshold, sample_size)
fc_data_file = './abide_fc_data_fisher_z.csv'

In [27]:
fcData = read_fc_data_file(fc_data_file)

,subject,func_perc_fd,AGE_YRS,SEX,DX_GROUP,#2001_#2002,#2001_#2101,#2001_#2102,#2001_#2111,#2001_#2112,...,#9130_#9150,#9130_#9160,#9130_#9170,#9140_#9150,#9140_#9160,#9140_#9170,#9150_#9160,#9150_#9170,#9160_#9170,func_perc_quart
1,50003,67.164179,24.0,1,1,1.373414,0.409453,0.789834,0.455512,0.471286,...,0.940071,0.798582,0.471726,0.857586,0.650342,0.592167,0.843476,0.599385,0.875041,3
2,50004,14.427861,19.0,1,1,0.824020,0.481031,0.258819,0.246055,0.306216,...,0.334438,0.234090,0.130916,0.632359,0.421152,-0.029851,0.866840,-0.061741,0.027180,2
3,50005,10.945274,13.0,2,1,1.135987,1.116633,0.944885,0.942790,0.780848,...,0.799417,0.778156,0.370868,0.590871,0.558449,0.262422,0.525498,0.256752,0.203965,2
4,50006,1.492537,13.0,1,1,1.456934,0.634692,0.303313,0.166852,-0.191597,...,0.420481,0.061504,0.391978,0.459101,0.095503,0.320199,0.121727,0.198423,0.114551,1
5,50007,18.905473,17.0,1,1,0.682998,0.959670,0.642405,0.977074,0.486507,...,0.934691,0.486346,0.626130,0.886814,0.432660,0.455022,0.656619,0.483390,0.405080,3


In [28]:
strata, labels, features = ppc_fc_data(fcData, age_range, motion_threshold)

364 364 364


In [66]:
from sklearn.cross_validation import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(strata, n_iter = oos_iter, test_size = sample_size, train_size = sample_size)

print("Running {} iterations of {} cv'd {} classification\n".format(oos_iter, cvmethod, classifier) + 
      "Each N = {}, Motion thresh = {}".format(sample_size, motion_threshold))

Running 10 iterations of sss cv'd svc classification
Each N = 30, Motion thresh = 5


In [52]:
for i, (train_index, test_index) in enumerate(sss):
    train_features, train_labels = features[train_index, :], labels[train_index]
    test_features, test_labels = features[test_index, :], labels[test_index]
    ifname_prefix = fname_prefix + '_i{:03d}'.format(i)
    aCLF = trainModel(train_features, train_labels, train_labels, 
                      modelDir=modelDir, fname_prefix=ifname_prefix, 
                      classifier = classifier,  
                      cv = True, cvmethod = cvmethod, 
                      n_iter = sss_n_iter, k = skf_n_folds, 
                      sparse = True, saveData = True, n_jobs=4)
    accuracy = testModel(test_features, test_labels, clf = aCLF, 
                         fname_prefix = ifname_prefix, 
                         outputDir = outputDir, saveData=True)

sss_svc_mt5_n30_i000
(array([1, 2]), array([12, 18]))
30 30 60
30 30
Training accuracy: 1.000000
Saving model weights, training accuracy, and model
Writing data to ./cv_models/test/sss_svc_mt5_n30_i000*
Testing accuracy: 0.666667
Saving test accuracy and predictions to ./cv_output/test/sss_svc_mt5_n30_i000*
sss_svc_mt5_n30_i001
(array([1, 2]), array([11, 19]))
30 30 60
30 30
Training accuracy: 0.633333
Saving model weights, training accuracy, and model
Writing data to ./cv_models/test/sss_svc_mt5_n30_i001*
Testing accuracy: 0.600000
Saving test accuracy and predictions to ./cv_output/test/sss_svc_mt5_n30_i001*
